In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [ ]:
fhm = pd.ExcelFile('https://www.arcgis.com/sharing/rest/content/items/b5e7488e117749c19881cce45db13f7e/data')

fhm.sheet_names

In [ ]:
df =pd.read_excel('https://www.arcgis.com/sharing/rest/content/items/b5e7488e117749c19881cce45db13f7e/data',sheet_name='Antal avlidna per dag',
                 parse_dates=True,skipfooter=1)

In [ ]:
df


In [ ]:
df.set_index('Datum_avliden',inplace=True)

In [ ]:
df

In [ ]:
df.plot(figsize=(18,12))
plt.grid(True)

In [ ]:
df_weekly = df.resample('W').sum()

In [ ]:
df_weekly

In [ ]:
df_weekly[:-1].plot(style='o--',figsize=(18,12))
plt.grid(True)

In [ ]:
conf_df = pd.read_excel('https://www.arcgis.com/sharing/rest/content/items/b5e7488e117749c19881cce45db13f7e/data',sheet_name='Antal per dag region',
                       usecols=[0,1],index_col=0,parse_dates=True)

In [ ]:
conf_df

In [ ]:
conf_weekly=conf_df.resample('W').sum()
conf_weekly

In [ ]:
conf_weekly[:-1].plot(style='o--',figsize=(18,12))
plt.grid(True)

In [ ]:
icu = fhm.parse('Antal intensivvårdade per dag', parse_dates=True,
               index_col=0)

icu

In [ ]:
icu.plot(figsize=(18,12))

In [ ]:
icu_weekly = icu.resample('W').sum()
icu_weekly

In [ ]:
icu_weekly[:-1].plot(figsize=(18,12),style='o--')

In [ ]:
scb = pd.ExcelFile('https://scb.se/hitta-statistik/statistik-efter-amne/befolkning/befolkningens-sammansattning/befolkningsstatistik/pong/tabell-och-diagram/preliminar-statistik-over-doda/')

In [ ]:
scb.sheet_names

In [ ]:
scb_daily = scb.parse('Tabell 1',skiprows=6)
scb_daily.index = pd.date_range('2020-01-01' , '2021-01-01')
scb_daily = scb_daily.drop('DagMånad',axis=1)
scb_daily = scb_daily.loc[:,'2015' : '2015-2019']
unknowns = scb_daily.iloc[-1]
scb_daily = scb_daily.iloc[:-1]
scb_daily

In [ ]:
knowns_sum = scb_daily.sum()
knowns_sum

In [ ]:
daily_pct = scb_daily / knowns_sum
daily_pct

In [ ]:
scb_daily_corr = scb_daily + unknowns * daily_pct
scb_daily_corr

In [ ]:
corr_diff = scb_daily_corr - scb_daily
corr_diff

In [ ]:
corr_diff.plot(figsize=(18,12))

In [ ]:
scb_daily_corr.sum()

In [ ]:
baseline = scb_daily['2015-2019']
baseline

In [ ]:
pct_change = scb_daily_corr.div(baseline,axis=0)
pct_change

In [ ]:
pct_change.plot(y='2020',figsize=(18,12))
plt.axhline(1,ls='dashed',color='k')

In [ ]:
weekly_change = pct_change.groupby(pct_change.index.week).mean()
weekly_change

In [ ]:
weekly_change.plot(figsize=(18,12),y='2020',style='o--')
plt.axhline(1,ls='dashed',color='k')

In [ ]:
cum_growth = scb_daily_corr.cumsum().div(baseline.cumsum(),axis=0)
cum_growth

In [ ]:
cum_growth.plot(y='2020', figsize=(18,12))
plt.axhline(1,ls='dashed',color='k')

In [ ]:
weekly_dead = scb_daily.groupby(scb_daily.index.week).sum()
weekly_dead

In [ ]:
weekly_cum_growth = weekly_dead['2020'].cumsum().div(weekly_dead['2015-2019'].cumsum())

weekly_cum_growth

In [ ]:
weekly_cum_growth.plot(figsize=(18,12),style='o--')
plt.axhline(1,ls='dashed',color='k')

In [ ]:
monthly_dead = scb_daily.groupby(scb_daily.index.month).sum()
monthly_dead

In [ ]:
monthly_cum_growth = monthly_dead['2020'].cumsum().div(monthly_dead['2015-2019'].cumsum())
monthly_cum_growth

In [ ]:
monthly_cum_growth.plot(figsize=(18,12),style='o--')
plt.axhline(1,ls='dashed',color='k')

In [ ]:
df_age = scb.parse('Tabell 7',skiprows=8,usecols=range(0,19))
df_age

In [ ]:
df_age = df_age.iloc[:-3,:]
df_age

In [ ]:
cols = [64,79,89,100]

In [ ]:
def add_unknowns(df):
    unknowns=df.iloc[-1,:]
    knowns = df.iloc[:-1,:]
    pct = knowns / knowns.sum()
    corr = knowns + unknowns * pct
    idx = pd.date_range('2020-01-01',freq='W',periods=53)
    corr.index=idx
    
    return corr

In [ ]:
df_age_male_base=df_age.iloc[:,2:6]
df_age_female_base=df_age.iloc[:,6:10]
df_age_male_base.columns=cols
df_age_female_base.columns=cols
df_age_male_base

In [ ]:
male_base = add_unknowns(df_age_male_base)
female_base = add_unknowns(df_age_female_base)

In [ ]:
df_age_male_base.sum(axis=1).sum()

In [ ]:
df_age_female_base

In [ ]:
df_age_male_20 = df_age.iloc[:,11:15]
df_age_male_20.columns=cols
df_age_male_20

In [ ]:
df_age_female_20 = df_age.iloc[:,15:]
df_age_female_20.columns=cols
df_age_female_20

In [ ]:
male_20 = add_unknowns(df_age_male_20)
female_20 = add_unknowns(df_age_female_20)

In [ ]:
age_both_base = male_base + female_base
age_both_20 = male_20 + female_20
age_both_20

In [ ]:

last_week = age_both_20.where(age_both_20 != 0).dropna().index[-1]
last_week
last_good_week = last_week - pd.DateOffset(weeks=2)
last_good_week

In [ ]:
age_both_20.sum(axis=1).sum()

In [ ]:
age_both_base.sum(axis=1).sum()

In [ ]:
weekly_growth = age_both_20 / age_both_base

In [ ]:
weekly_growth.plot(figsize=(18,12),style='o--')

In [ ]:
weekly_cum_growth = age_both_20.cumsum() / age_both_base.cumsum()

In [ ]:
weekly_cum_growth.plot(figsize=(18,12),style='o--')

In [ ]:
import requests
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860'

json_data = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:Ålder1årA",
        "values": [
          "0",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27",
          "28",
          "29",
          "30",
          "31",
          "32",
          "33",
          "34",
          "35",
          "36",
          "37",
          "38",
          "39",
          "40",
          "41",
          "42",
          "43",
          "44",
          "45",
          "46",
          "47",
          "48",
          "49",
          "50",
          "51",
          "52",
          "53",
          "54",
          "55",
          "56",
          "57",
          "58",
          "59",
          "60",
          "61",
          "62",
          "63",
          "64",
          "65",
          "66",
          "67",
          "68",
          "69",
          "70",
          "71",
          "72",
          "73",
          "74",
          "75",
          "76",
          "77",
          "78",
          "79",
          "80",
          "81",
          "82",
          "83",
          "84",
          "85",
          "86",
          "87",
          "88",
          "89",
          "90",
          "91",
          "92",
          "93",
          "94",
          "95",
          "96",
          "97",
          "98",
          "99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2014",
          "2015",
          "2016",
          "2017",
          "2018",
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(url,json=json_data)
r.status_code

In [ ]:
json = r.json()

json['data'][0]

In [ ]:
rows = len(json['data'])

data_list = []

for r in range(rows):
    rec = json['data']
    data_list.append([rec[r]['key'][0],rec[r]['key'][1],rec[r]['key'][2],rec[r]['values'][0]])
    
data_list

In [ ]:
pop_df = pd.DataFrame(data_list,columns=['age','gender','year','pop'])
pop_df['pop'] = pop_df['pop'].astype(int)
pop_df['age'] = pop_df['age'].apply(lambda x : x.replace('+','')).astype(int)

pop_df

In [ ]:
pop_age_tot = pop_df.groupby(['year','age']).sum()
pop_age_tot

In [ ]:
pop_age_tot_flat = pop_age_tot.unstack()
pop_age_tot_flat = pop_age_tot_flat.droplevel(0,axis=1)
pop_age_tot_flat

In [ ]:
grp_64 = pop_age_tot_flat.loc[:,:64].sum(axis=1)
grp_79 = pop_age_tot_flat.loc[:,65:79].sum(axis=1)
grp_89 = pop_age_tot_flat.loc[:,80:89].sum(axis=1)
grp_100 = pop_age_tot_flat.loc[:,90:].sum(axis=1)
grp_100

In [ ]:
pop_age_grps = pd.concat([grp_64,grp_79,grp_89,grp_100],axis=1)
pop_age_grps.columns = cols
pop_age_grps

In [ ]:
pop_age_grps.plot(kind='bar',stacked=True,figsize=(18,12))

In [ ]:
baseline_pop = pop_age_grps.loc['2014':'2018'].mean()
baseline_pop

In [ ]:
twenty_twenty_pop = pop_age_grps.loc['2019']
twenty_twenty_pop.name = '2020'
twenty_twenty_pop

In [ ]:
baseline_weekly_mortality = age_both_base / baseline_pop
twenty_twenty_weekly_mortality = age_both_20 / twenty_twenty_pop

twenty_twenty_weekly_mortality

In [ ]:
weekly_mortality = pd.concat([baseline_weekly_mortality,twenty_twenty_weekly_mortality],axis=1)
weekly_mortality.columns = ['15-19_age_grp_64',
                           '15-19_age_grp_79',
                           '15-19_age_grp_89',
                           '15-19_age_grp_90+',
                           '20_age_grp_64',
                           '20_age_grp_79',
                           '20_age_grp_89',
                           '20_age_grp_90+']

weekly_mortality

In [ ]:
weekly_mortality.plot(figsize=(18,12),style='o--')

In [ ]:
weekly_growth = twenty_twenty_weekly_mortality / baseline_weekly_mortality
weekly_growth

In [ ]:
title = 'SWEDEN weekly mortality growth factor 2020 YTD {} vs baseline 2015-2019'.format(last_good_week.date())

weekly_growth[:last_good_week].plot(figsize=(18,12),style='o--',title=title)
plt.axhline(1,ls='dashed',color='k')
plt.ylabel('growth factor')

In [ ]:
weekly_cum_growth = twenty_twenty_weekly_mortality.cumsum() / baseline_weekly_mortality.cumsum()
weekly_cum_growth

In [ ]:
title = 'SWEDEN weekly cumulative mortality growth factor 2020 YTD {} vs baseline 2015-2019\nDataSource : SCB.se'.format(last_good_week.date())

weekly_cum_growth[:last_good_week].plot(figsize=(18,12),style='o--',title=title)
plt.axhline(1,ls='dashed',color='k')
plt.ylabel('growth factor')
plt.legend(['-64','65-79','80-89','90+'])
plt.savefig('scb_weekly_growth_factors_vs_15_19.jpg',format='jpg')

In [ ]:
ytd_deaths_20 = age_both_20.loc[:last_good_week].sum()
ytd_deaths_base = age_both_base.loc[:last_good_week].sum()
ytd_deaths_20


In [ ]:
ytd_deaths = pd.concat([ytd_deaths_base,ytd_deaths_20],axis=1)
ytd_deaths.columns = ['15-19-base','2020']
ytd_deaths

In [ ]:
ytd_deaths.plot(kind='bar')

In [ ]:
ytd_mortality = weekly_mortality.loc[:last_good_week].sum()
ytd_mortality_combined = pd.DataFrame({'15-19-base' :ytd_mortality.iloc[:4].values,
                             '2020' : ytd_mortality.iloc[4:].values})

ytd_mortality_combined.index = [64,79,89,100]
ytd_mortality_combined

In [ ]:
title = 'SWEDEN age group mortality YTD {}\nDataSource : SCB.se'.format(last_good_week.date())

ytd_mortality_combined.plot(kind='bar',figsize=(18,12),logy=True,title=title)

_= plt.xticks(range(len(ytd_mortality_combined)),['-64','65-79','80-89','90+'])
plt.ylabel('mortality [log scale]')
plt.xlabel('age group')
plt.savefig('swe_age_grp_mortality_ytd.jpg',format='jpg')

In [ ]:
ytd_mortality_combined_growth = ytd_mortality_combined['2020'] / ytd_mortality_combined['15-19-base']
ytd_mortality_combined_growth

In [ ]:
title='SWEDEN age grp mortality growth factors 2020 vs baseline 2015-2019, YTD {}\nDataSource : scb.se'.format(
last_good_week.date())
plt.ylabel('growth factor')
plt.xlabel('age group')
plt.axhline(1,ls='dashed',color='k')
ytd_mortality_combined_growth.plot(kind='bar',figsize=(18,12),color='red',title=title)
_=plt.xticks(range(len(ytd_mortality_combined_growth)),['-64','65-79','80-89','90+'])
plt.yticks(np.arange(0,1.1,0.05))
plt.savefig('swe_age_grp_mortality_growth_ytd.jpg',format='jpg')

In [ ]:
fhm_age = fhm.parse('Totalt antal per åldersgrupp')
fhm_age

In [ ]:
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101I/DodaFodelsearK'

json_data = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:RegionRiket99",
        "values": [
          "00"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:Ålder1årA",
        "values": [
          "0",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27",
          "28",
          "29",
          "30",
          "31",
          "32",
          "33",
          "34",
          "35",
          "36",
          "37",
          "38",
          "39",
          "40",
          "41",
          "42",
          "43",
          "44",
          "45",
          "46",
          "47",
          "48",
          "49",
          "50",
          "51",
          "52",
          "53",
          "54",
          "55",
          "56",
          "57",
          "58",
          "59",
          "60",
          "61",
          "62",
          "63",
          "64",
          "65",
          "66",
          "67",
          "68",
          "69",
          "70",
          "71",
          "72",
          "73",
          "74",
          "75",
          "76",
          "77",
          "78",
          "79",
          "80",
          "81",
          "82",
          "83",
          "84",
          "85",
          "86",
          "87",
          "88",
          "89",
          "90",
          "91",
          "92",
          "93",
          "94",
          "95",
          "96",
          "97",
          "98",
          "99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2014",
          "2015",
          "2016",
          "2017",
          "2018",
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(url,json=json_data)
r.status_code

In [ ]:
json = r.json()
json['data'][0]

In [ ]:
nr_recs = len(json['data'])
data_list = []

for rec in range(nr_recs):
    data_list.append([json['data'][rec]['key'][1],json['data'][rec]['key'][2],
                     json['data'][rec]['key'][3],json['data'][rec]['values'][0]])
    


In [ ]:
dead = pd.DataFrame(data_list,columns=['age','gender','year','dead'])
dead['dead'] = dead['dead'].astype(int)
dead['age'] = dead['age'].str.replace('+','').astype(int)
dead

In [ ]:
dead = dead.groupby(['year','age']).sum()
dead

In [ ]:
dead = dead.unstack()
dead = dead.droplevel(0,axis=1)
dead

In [ ]:
dead_0_64 = dead.loc[:,0:64].sum(axis=1)
dead_65_79 = dead.loc[:,65:79].sum(axis=1)

print (dead_0_64)
print()
print (dead_65_79)


In [ ]:
full_year_dead = dead.sum(axis=1)
full_year_dead

In [ ]:
pct_0_64 = dead_0_64 / full_year_dead
pct_65_79 = dead_65_79 / full_year_dead
print (pct_0_64)
print (pct_65_79)

In [ ]:
dead_0_9 = dead.loc[:,:9].sum(axis=1)
dead_10_19 = dead.loc[:,10:19].sum(axis=1)
dead_20_29 = dead.loc[:,20:29].sum(axis=1)
dead_30_39 = dead.loc[:,30:39].sum(axis=1)
dead_40_49 = dead.loc[:,40:49].sum(axis=1)
dead_50_59 = dead.loc[:,50:59].sum(axis=1)
dead_60_69 = dead.loc[:,60:69].sum(axis=1)
dead_70_79 = dead.loc[:,70:79].sum(axis=1)
dead_80_89 = dead.loc[:,80:89].sum(axis=1)
dead_90_ = dead.loc[:,90:].sum(axis=1)

dead_90_

In [ ]:
dead_bin_10y = pd.concat([dead_0_9,dead_10_19,dead_20_29,dead_30_39,dead_40_49,
                         dead_50_59,dead_60_69,dead_70_79,dead_80_89,dead_90_],axis=1)

dead_bin_10y.columns = range(0,100,10)

dead_bin_10y

In [ ]:
dead_bin_10y_base = dead_bin_10y.loc['2014':'2018'].mean()
dead_bin_10y_base

In [ ]:
acd_cum_ratio = dead_bin_10y_base.cumsum() / dead_bin_10y_base.sum()
ax = acd_cum_ratio.plot(style='o--')
acd_cum_ratio

In [ ]:
fhm_dead = fhm_age['Totalt_antal_avlidna']
fhm_dead = fhm_dead[:-1]
fhm_dead

In [ ]:
fhm_cum_ratio = fhm_dead.cumsum() / fhm_dead.sum()
fhm_cum_ratio.index = range(0,100,10)
fhm_cum_ratio

In [ ]:
title = 'SWEDEN cumulative deaths per age group : distributions for All-Cause baseline 15-19 and Covid registered deaths 2020'\
+ ' YTD {}\nDataSource : scb.se, FHM'.format(last_good_week.date())

ax = acd_cum_ratio.plot(figsize=(18,12),style='go--',title=title)
fhm_cum_ratio.plot(ax=ax,style='ro--')
plt.xticks(acd_cum_ratio.index)

plt.legend(['All Cause Deaths avg 2015-2019','Covid-reg. Deaths 2020'])

plt.ylabel('ratio of deaths, [All Cause,Covid]')
plt.xlabel('age group [10 year bins]')
#plt.yscale('log')
_= plt.yticks(np.arange(0,1.05,0.05))

In [ ]:
acd_cum_age_grp_growth = acd_cum_ratio / acd_cum_ratio.shift()
fhm_cum_age_grp_growth = fhm_cum_ratio / fhm_cum_ratio.shift()

cum_age_grp_growth = pd.concat([acd_cum_age_grp_growth,fhm_cum_age_grp_growth],axis=1)
cum_age_grp_growth.columns=['All_Cause_Deaths','Covid_reg_Deaths']
cum_age_grp_growth

In [ ]:
cum_age_grp_growth.plot(figsize=(18,12),style='o--')

In [ ]:
fhm_all = pd.concat([df,icu,conf_df],axis=1)
fhm_all.columns=['Covid_deaths','Covid_ICUs','Covid_Pos']
print (fhm_all)
fhm_all = fhm_all.loc[:last_good_week]
fhm_all

In [ ]:
fhm_dist = fhm_all / fhm_all.sum()
fhm_dist

In [ ]:
fhm_cum_dist = fhm_dist.cumsum()
fhm_cum_dist

In [ ]:
fhm_cum_dist.plot(figsize=(18,12),color=['r','g','b'])

In [ ]:
fhm_age
fhm_age = fhm_age.drop(10)
fhm_age

In [ ]:
fhm_age.index = range(0,100,10)
fhm_age = fhm_age.drop('Åldersgrupp',axis=1)
fhm_age.columns=['Covid_Pos','Covid_ICUs','Covid_Deaths']

fhm_age

In [ ]:
fhm_age_dist = fhm_age / fhm_age.sum()
fhm_age_dist

In [ ]:
fhm_age_cum_dist = fhm_age_dist.cumsum()
fhm_age_cum_dist

In [ ]:
title = 'SWEDEN Covid : cumulative distribution of events over age groups YTD {}'.format(last_good_week.date())

ax = fhm_age_cum_dist.plot(figsize=(18,12),style=['go--','bo--','ro--'],title=title)

acd_cum_ratio.plot(ax=ax,style='ko--')

plt.xticks(fhm_age_cum_dist.index,['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90+'])
plt.xlabel('age group')
plt.ylabel('fraction of total')
_= plt.yticks(np.arange(0,1.05,0.05))
plt.legend(['Covid_Pos','Covid_ICUs','Covid_Deaths','15-19_All_Cause_Deaths'])
plt.savefig('swe_covid_cum_dist.jpg',format='jpg')

In [ ]:
weekly_baseline = weekly_mortality.loc[:,'15-19_age_grp_64':'15-19_age_grp_90+']
weekly_baseline.columns=[64,79,89,100]

weekly_baseline

In [ ]:
age_grp_mort = weekly_baseline.sum()
age_grp_mort

In [ ]:
pop_age_grps

In [ ]:
age_grp_strat_dead = age_grp_mort * pop_age_grps
age_grp_strat_dead

In [ ]:
strat_dead = age_grp_strat_dead.sum(axis=1)
strat_dead

In [ ]:
strat_dead / pop_age_grps.sum(axis=1)

In [ ]:
scb_daily_corr

In [ ]:
common_pop = pop_df.groupby(['year'])['pop'].sum()
common_pop.index = range(2015,2021)
common_pop

In [ ]:
total_abs_deaths = scb_daily_corr.sum()
total_abs_deaths

In [ ]:
mort_base = total_abs_deaths.loc['2015-2019'] / common_pop.loc['2015':'2019'].mean()
mort_base

In [ ]:
normal_2020_deaths = mort_base * common_pop.loc[2020]
normal_2020_deaths

In [ ]:
scb_daily_corr.loc[:last_good_week].sum()

In [ ]:
current_2020_deaths = scb_daily_corr.loc[:last_good_week].sum()
current_2020_deaths

In [ ]:
remaining_deaths = normal_2020_deaths - current_2020_deaths.loc['2020']
remaining_deaths

In [ ]:
scb_daily_corr.loc[:last_good_week].resample('M').sum()

In [ ]:
forecast_deaths = scb_daily_corr.sum()
forecast_deaths['2020'] = normal_2020_deaths
forecast_deaths

In [ ]:

common_pop.index = ['2015','2016','2017','2018','2019','2020']
common_pop['2015-2019'] = common_pop.loc['2015' : '2019'].mean()
forecasted_uncorrected_nov = forecast_deaths / common_pop

In [ ]:
print ('last_data_date:',(scb_daily_corr.loc[scb_daily_corr['2020'] != 0]).index[-1])
scb_daily_corr.resample('M').sum()

In [ ]:
nov_2020 = scb_daily_corr.resample('M').sum()
nov_2020_ytd = nov_2020.loc['2020-11-30','2020']
nov_2020_ytd

In [ ]:
# assuming 1:St week after month change the numbers are 85%, after second 95%, of final
nov_2020_ytd_corr = nov_2020_ytd / 0.85
nov_2020_ytd_corr

In [ ]:
totals_inc_oct_2020 = scb_daily_corr.loc[:'2020-10-31','2020'].resample('M').sum()
totals_inc_oct_2020 = totals_inc_oct_2020.sum()
totals_inc_oct_2020


In [ ]:
left_for_dec = normal_2020_deaths - (totals_inc_oct_2020 + nov_2020_ytd_corr)
left_for_dec

In [ ]:
base_dec = (scb_daily_corr.resample('M').sum()).loc['2020-12-31','2015-2019']

In [ ]:
forecasted_corr_total = totals_inc_oct_2020 + nov_2020_ytd_corr + base_dec
forecasted_corr_total

In [ ]:
forecasted_corr_total / common_pop['2020']

In [ ]:
forecasted_uncorrected_nov